### 1) 안전점수: Lane Mark가 항상 0으로 계산되던 버그 수정

변경사항
- 안전점수 계산에서 Lane Mark 클래스 이름을 잘못 조회해서 lane_px가 항상 0으로 떨어지던 문제를 수정
- 안전점수 계산 함수/셀에서 lane_px 계산하는 부분 (px(...) 호출부) 수정

기존
lane_px = px('Lanemark')

변경
lane_px = px('Lane Mark')

원인
- 클래스 이름이 EDA/학습 쪽에서는 "Lane Mark"(공백 포함)인데, 안전점수 로직에서는 "Lanemark"(공백 없음)을 찾고 있었음

그 결과 name_to_idx.get("Lanemark")가 실패해서 -1 또는 0 처리로 이어지고, 차선 픽셀 수가 항상 0 → 차선 가시성 평균이 0으로 고정되는 현상이 발생

### 2) SegFormer 추론(infer_pred_mask) 호출 방식 수정

변경사항
- infer_pred_mask()에서 SegFormer 분기일 때 model(x)처럼 호출하던 부분을 HuggingFace SegFormer 표준 호출로 변경
- 정성평가/후처리 파트의 infer_pred_mask(model, img_tensor, device, model_type) 함수 내부 segformer 분기

기존
if model_type == "segformer":
    logits = model(x)

변경
if model_type == "segformer":
    out = model(pixel_values=x)
    logits = out.logits

원인
- SegFormer는 보통 model(pixel_values=...)로 호출하고 결과에서 .logits를 꺼내야 함
- model(x)로 하면 출력 타입/키가 달라서 런타임 에러 또는 잘못된 logits 사용 가능성이 있음
- 정성평가/안전점수 계산에 쓰는 “추론 단계”가 안정적으로 돌아가도록 수정

### 3) “Stratified Split” 설명과 실제 코드 불일치 정리

변경사항
- 문서/마크다운에 “Stratified Split”이라고 적혀 있던 표현을 실제 구현(랜덤 분할)에 맞게 정리
- 데이터 분할 설명 마크다운(보고서 서술) 부분

기존
“Stratified Split”

변경
“Random Train/Validation Split” (또는 “Random Split”)

원인
- 코드에서 train_test_split(..., stratify=...)가 없는데 “Stratified”라고 써 있으면 결과 신뢰도/재현성 설명이 틀어짐
- 세그멘테이션은 “이미지 단위 stratify”가 애매해서(픽셀 분포 기준 등) 급하게 억지로 넣는 것보다, 현실적으로는 random split로 명확히 적는 편이 안전